# Preparing the Data for Training

### This Jupyter Notebook replicated the preprocessing steps on the original paper for better understanding

The architecture of DeepSignalingFlow forces us to create seperate networks for each available dataset depending on the number of genes and drug combinations. We will thus start by only preprocessing the NCI ALMANAC Dataset.

To prepare we need to download the following Datasets:

NCI ALMANAC [Download](https://wiki.nci.nih.gov/spaces/NCIDTPdata/pages/338237347/NCI-ALMANAC) <br>
KEGG [Download](https://www.genome.jp/kegg/) <br>
DrugBank [Download](https://go.drugbank.com/) <br>
Cell Model Passports RNA-Seq [Download](https://cog.sanger.ac.uk/cmp/download/rnaseq_20191101.zip) <br>
Cell Model Passports CNV [Download](https://cog.sanger.ac.uk/cmp/download/cnv_20191101.zip) <br>


